In [46]:
import torch
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F



In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
 ])

In [4]:
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2)


100%|██████████| 170M/170M [06:37<00:00, 429kB/s]    


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
image, label = train_data[0]

In [ ]:
#image.size()

torch.Size([3, 32, 32])

In [32]:
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

In [22]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5) # 32- 5 = 27//2 +1
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fully_connected1 = nn.Linear(24*5*5, 120)
        self.fully_connected2 = nn.Linear(120, 84)
        self.fully_connected3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fully_connected1(x))
        x = F.relu(self.fully_connected2(x))
        x = self.fully_connected3(x)
        return x


In [24]:
neural_network = ImageClassifier()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(neural_network.parameters(), lr=0.01, momentum = 0.9)

In [42]:
for epoch in range(30):
    print(f"Training epoch {epoch}....")

    running_loss = 0.0

    for i, image in enumerate(train_loader):
        inputs, labels = image
        optimizer.zero_grad()
        outputs = neural_network(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Loss: {running_loss / len(train_loader):.4f}")
    


Training epoch 0....
Loss: 0.4053
Training epoch 1....
Loss: 0.4054
Training epoch 2....
Loss: 0.4053
Training epoch 3....
Loss: 0.4053
Training epoch 4....
Loss: 0.4053
Training epoch 5....
Loss: 0.4053
Training epoch 6....
Loss: 0.4053
Training epoch 7....
Loss: 0.4054
Training epoch 8....
Loss: 0.4054
Training epoch 9....
Loss: 0.4053
Training epoch 10....
Loss: 0.4054
Training epoch 11....
Loss: 0.4054
Training epoch 12....
Loss: 0.4053
Training epoch 13....
Loss: 0.4053
Training epoch 14....
Loss: 0.4057
Training epoch 15....
Loss: 0.4054
Training epoch 16....
Loss: 0.4053
Training epoch 17....
Loss: 0.4053
Training epoch 18....
Loss: 0.4053
Training epoch 19....
Loss: 0.4053
Training epoch 20....
Loss: 0.4053
Training epoch 21....
Loss: 0.4053
Training epoch 22....
Loss: 0.4054
Training epoch 23....
Loss: 0.4053
Training epoch 24....
Loss: 0.4053
Training epoch 25....
Loss: 0.4055
Training epoch 26....
Loss: 0.4053
Training epoch 27....
Loss: 0.4053
Training epoch 28....
Loss: 0.

In [43]:
torch.save(neural_network.state_dict(),'trained_model.pth')

In [44]:
neural_network = ImageClassifier()
neural_network.load_state_dict(torch.load("trained_model.pth", weights_only=True))

<All keys matched successfully>

In [41]:
correct = 0
total = 0

neural_network.eval()

with torch.no_grad():
    for image in test_loader:
        images, labels = image
        outputs = neural_network(images)
        __, predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 62.25%


In [45]:
new_transform = transforms.Compose([
    #transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5, 0.5))
])


def load_image(image_path):
    image = Image.open(image_path).convert("RGB") # Force 3 cha
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['ship.png', 'plane.png']
images = [load_image(img) for img in image_paths]

with torch.no_grad():
    for image in images:
        output = neural_network(image)  
        __, predicted = torch.max(output, 1)
        print(f"Prediction: {class_names[predicted.item()]}")

Prediction: ship
Prediction: airplane
